In [19]:
import importlib, os, gsw, gc

import pandas as pd
import xarray as xr
import numpy as np
from scipy.interpolate import interp1d
from scipy.optimize import fmin
from tqdm import tqdm

import SXBQ as sx

import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
%matplotlib widget

importlib.reload(sx)

<module 'SXBQ' from 'C:\\Users\\johan\\OneDrive\\Bureau\\PFE-Goteborg\\Scripts\\SXBQ.py'>

# LOAD HYDROGRAPHIC DATA

In [20]:
data = sx.sxdf()
data.data = pd.read_parquet('C://Users/johan/OneDrive/Bureau/PFE-Goteborg/results.pqt')

top_mounted = True

def delimitateProfiles():
    #data.data.sort_values('Timestamp', ignore_index=True, inplace=True)
    data.median_resample()

    _gd = np.isfinite(data.data['diveNum'].values)
    _, _tmp = np.unique( np.round(data.data['diveNum'].values[_gd]) , return_inverse=True)

    data.data.loc[_gd,'diveNum'] = np.round(_tmp+1)
    data.data['diveNum'] = data.data['diveNum'].interpolate('nearest')
        
    data.data['profileNum'] = data.data['diveNum'].values*2
    _tmp = data.data['NAV_RESOURCE'].interpolate('nearest').values
    ind = (_tmp == 100) | (_tmp == 110) | (_tmp == 116)
    data.data.loc[ind,'profileNum'] = data.data.loc[ind,'profileNum'] - 1

delimitateProfiles()

def rmsd(x):
    return np.sqrt(np.nanmean(x**2))

#data.data

# LOAD ADCP DATA

In [3]:
adcp_path = 'C://Users/johan/OneDrive/Bureau/PFE-Goteborg/Scripts/Data/ADCP/sea045_M33_A0'

header = ['Month','Day','Year','Hour','Minute','Second','Burst counter','Ensemble counter','Error code','Status code','Battery voltage','Soundspeed','Heading','Pitch','Roll','Pressure','Temperature','Analog input 1','Analog input 2']
ADCP = pd.read_table(adcp_path+'.sen',sep=r"\s+",names=header)
ADCP.insert(0,'time',pd.to_datetime(ADCP[['Year', 'Month', 'Day', 'Hour', 'Minute','Second']], utc=True, origin='unix', cache='False'))

ADCP.set_index('time', inplace=True)

ADCP = ADCP.to_xarray()

for beam in ['1','2','3','4']:
    ADCP['V'+beam] = (
                   ['time','bin'],
                   pd.read_csv(adcp_path+'.v'+beam, sep=r"\s+", header=None).iloc[:,2:]
                  )
    ADCP['C'+beam] = (
                   ['time','bin'],
                   pd.read_csv(adcp_path+'.c'+beam, sep=r"\s+", header=None).iloc[:,2:]
                  )

ADCP

<xarray.Dataset>
Dimensions:           (bin: 15, time: 183468)
Coordinates:
  * time              (time) object 1597909692438000000 ... 1600074055438000000
Dimensions without coordinates: bin
Data variables: (12/27)
    Month             (time) int64 8 8 8 8 8 8 8 8 8 8 8 ... 9 9 9 9 9 9 9 9 9 9
    Day               (time) int64 20 20 20 20 20 20 20 ... 10 10 10 10 14 14 14
    Year              (time) int64 2020 2020 2020 2020 ... 2020 2020 2020 2020
    Hour              (time) int64 7 7 7 7 7 7 7 7 7 ... 14 14 14 14 14 14 9 9 9
    Minute            (time) int64 48 48 48 48 48 49 49 49 ... 7 7 7 7 7 0 0 0
    Second            (time) float64 12.44 22.44 32.44 ... 35.44 45.44 55.44
    ...                ...
    V2                (time, bin) float64 0.127 -0.083 -0.627 ... 0.971 1.979
    C2                (time, bin) float64 48.0 0.0 1.0 0.0 ... 2.0 3.0 1.0 2.0
    V3                (time, bin) float64 0.0 -1.065 -1.168 ... 1.946 0.748
    C3                (time, bin) float64 56.0 2.0 3.0 1.0 ... 0.0 2.0 5.0 2.0
    V4                (time, bin) float64 0.1 2.003 -0.928 ... -2.223 1.973
    C4                (time, bin) float64 47.0 0.0 0.0 0.0 ... 3.0 0.0 1.0 2.0

# CALCULATE ADCP BIN DEPTH

In [4]:
def getADCPBinDepth(ADCP, bin_size, blanking_distance,direction='down',lat=55):
    # Retuns a new coordinate within the ADCP matrix of size ntime x nbin containing ADCP bin depths
    if top_mounted:
        direction = 1
    else:
        direction = -1

    bin_distance = blanking_distance + np.arange(len(ADCP.bin))*bin_size + 0.5*bin_size
    
    alpha = np.arccos(
                np.cos(np.deg2rad(22.5 + ADCP['Pitch'])) * # THIS IS WRONG
                np.cos(np.deg2rad(ADCP['Roll'])) 
            )
    adcp_depth = gsw.z_from_p(ADCP['Pressure'],lat)

    bin_depth = np.tile(adcp_depth, (len(ADCP.bin), 1)).T \
                - direction \
                * np.tile(bin_distance, (len(ADCP.time), 1)) \
                * np.tile(np.cos(alpha), (len(ADCP.bin), 1)).T \

    return ADCP.assign_coords({'bin_depth':(['time','bin'], bin_depth)})

ADCP = getADCPBinDepth(ADCP,2,0.2,55)

# SOUNDSPEED CORRECTION

In [5]:
# Not implemented yet

# 3 BEAM PITCH-DEPENDENT SOLUTION TO XYZ AND ENU CALCULATION

In [6]:
def calcXYZfrom3beam():
    def sin(x):
        return np.sin(np.deg2rad(x))
    def cos(x):
        return np.cos(np.deg2rad(x))

    a = 47.5 # Beam 1 and 3 angle from Z
    b = 25 # Beam 2 and 4 angle from Z

    xyz2beam = np.array([
        [sin(a),0,cos(a)],
        [0,-sin(b),cos(b)],
        [0,sin(b),cos(b)]
    ])

    beam2xyz = np.linalg.inv(xyz2beam)

    V_fore = beam2xyz @ np.array([
        ADCP['V1'].values.flatten(),
        ADCP['V2'].values.flatten(),
        ADCP['V4'].values.flatten()
        ])
    V_aft = beam2xyz @ np.array([
        ADCP['V3'].values.flatten(),
        ADCP['V2'].values.flatten(),
        ADCP['V4'].values.flatten()
        ])

    if rmsd(V_aft[1:,:]-V_fore[1:,:]) != 0:
        print('Something is wrong - abort and investigate...')

    X = np.reshape( V_fore[0,:] , (-1,15) )
    X2 = np.reshape( V_aft[0,:] , (-1,15) )
    
    plt.close('all')
    _ = plt.hist(X.flatten(),100,color='r',alpha=0.3)
    _ = plt.hist(-X2.flatten(),100,color='b',alpha=0.3)
    
    use_aft_on_climb = ADCP['Pitch'] > 0
    
    if top_mounted == True:
        print('Assuming ADCP is top mounted')
        X[~use_aft_on_climb,:] = -X2[~use_aft_on_climb,:]
        V_aft[1,:] = -V_aft[1,:]
    else:
        print('Assuming ADCP is bottom mounted')
        X[use_aft_on_climb,:] = -X2[use_aft_on_climb,:]
    
    
    _ = plt.hist(X.flatten(),100,color='g',alpha=0.3)
    
    ADCP['X'] = (['time','bin'], X )
    ADCP['Y'] = (['time','bin'], np.reshape( V_aft[1,:] , (-1,15) ) )
    ADCP['Z'] = (['time','bin'], np.reshape( V_aft[2,:] , (-1,15) ) )
    
    
calcXYZfrom3beam()
#gc.collect()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Assuming ADCP is top mounted


In [7]:
def calcENUfromXYZ():
    def M_xyz2enu(heading,pitch,roll):
        hh = np.pi*(heading-90)/180
        pp = np.pi*pitch/180
        rr = np.pi*roll/180
        
        _H = np.array([
            [np.cos(hh),np.sin(hh),0], 
            [-np.sin(hh),np.cos(hh),0], 
            [0,0,1]
        ])
        _P = np.array([
            [np.cos(pp), -np.sin(pp)*np.sin(rr), -np.cos(rr)*np.sin(pp)] ,
            [0, np.cos(rr), -np.sin(rr)] , 
            [ np.sin(pp), np.sin(rr)*np.cos(pp), np.cos(pp)*np.cos(rr)]
        ])
        
        _R = _H@_P
        return _R

    H = ADCP['Heading'].values
    P = ADCP['Pitch'].values
    R = ADCP['Roll'].values

    E = ADCP['X'].values.copy()
    N = ADCP['Y'].values.copy()
    U = ADCP['Z'].values.copy()

    r,c = np.shape(E)

    for i in tqdm(range(r)):
        XYZ2ENU = M_xyz2enu(H[i],P[i],R[i])
        for j in range(c):
            E[i,j], N[i,j], U[i,j] = XYZ2ENU @ [E[i,j], N[i,j], U[i,j]]

    ADCP['E'] = (['time','bin'], E )
    ADCP['N'] = (['time','bin'], N )
    ADCP['U'] = (['time','bin'], U )

calcENUfromXYZ()
gc.collect()

100%|███████████████████████████████████████████████████████████████████████| 183468/183468 [00:16<00:00, 11155.82it/s]


40

In [8]:
plt.figure()

PD = ADCP['Pitch'] < 0
PU = ADCP['Pitch'] > 0

plt.subplot(511)
_ = plt.hist(ADCP.isel(bin=0)['X'][PD].values,np.linspace(-1,1,100),color='r')
_ = plt.hist(ADCP.isel(bin=0)['X'][PU].values,np.linspace(-1,1,100),color='b')
plt.title('Glider moving forward so expect X negative')

plt.subplot(513)
_ = plt.hist(ADCP.isel(bin=0)['U'][PD].values,np.linspace(-1,1,100),color='r')
plt.title('Glider diving so expect U positive')

plt.subplot(515)
_ = plt.hist(ADCP.isel(bin=0)['U'][PU].values,np.linspace(-1,1,100),color='b')
plt.title('Glider climbing so expect U negative')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Glider climbing so expect U negative')

In [9]:
def rmsd(x):
    return np.sqrt(np.nanmean(x**2))
def smooth(x,N):
    return np.convolve(x, np.ones(N)/N, mode='same')

%matplotlib widget
plt.close('all')

spd_xyz = np.sqrt(ADCP.isel(bin=0)['X']**2 + ADCP.isel(bin=0)['Y']**2 + ADCP.isel(bin=0)['Z']**2)

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
plt.plot(ADCP.time.values.astype('float'),smooth(spd_xyz,5), alpha=0.3, color='g',label="ADCP")
plt.plot(data.data['Timestamp'].values.astype('float'),data.data.speed,alpha=0.9,color='k',label="steady flight model")
plt.title("speed of glider")
plt.xlabel("t [s]")
plt.ylabel("v [m/s]")
plt.legend()

t = np.nanpercentile(data.data['Timestamp'].values.astype('float'),[0,100])

def update(x=0):
    ax.set_xlim([t[0] + x*step , t[0] + x*step + step])
    fig.canvas.draw_idle()

ax.set_ylim([0,1.2])
step = np.diff(t)/100

update(30)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Steady flight validity model

In [25]:
%matplotlib widget
plt.close('all')


a=np.diff(data.data.speed)
L=2.7 #length of glider
V=np.nanmean(data.data.speed) #Average speed of drone
v=np.nan_to_num(data.data.speed)
a_max=V*V/(10*L)
print(a_max)

_u = np.remainder(data.data.profileNum,2) == 0
_d = np.remainder(data.data.profileNum,2) == 1
_u=_u[0:len(a)]
_d=_d[0:len(a)]

T=data.data['Timestamp'].values.astype('float')[0:len(a)]

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
plt.plot([ADCP.time.values.astype('float')[0],ADCP.time.values.astype('float')[-1]],[a_max,a_max], alpha=1, color='g',label="domain of validity")
plt.plot([ADCP.time.values.astype('float')[0],ADCP.time.values.astype('float')[-1]],[-a_max,-a_max], alpha=1, color='g',label="domain of validity")
#plt.plot(T[_d],a[_d],alpha=0.9,color='r',label="steady flight model (downcast)")
#plt.plot(T[_u],a[_u],alpha=0.9,color='b',label="steady flight model (upcast)")
plt.scatter(T[_d],a[_d],1,'r', alpha=1,label='downcast')
plt.scatter(T[_u],a[_u],1,'b', alpha=1,label='upcast')

plt.title("Acceleration of glider")
plt.xlabel("t [s]")
plt.ylabel("a [m2/s]")
plt.legend()

t = np.nanpercentile(data.data['Timestamp'].values.astype('float'),[0,100])

def update(x=0):
    ax.set_xlim([t[0] + x*step , t[0] + x*step + step])
    fig.canvas.draw_idle()

ax.set_ylim([-0.02,0.02])
step = np.diff(t)/100

update(30)

plt.show()

0.004851022078631626


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Search for errors

#### Upcast VS Downcast

In [24]:
_u = np.remainder(data.data.profileNum,2) == 0
_d = np.remainder(data.data.profileNum,2) == 1

def rmsd(x):
    return np.sqrt(np.nanmean(x**2))
def smooth(x,N):
    return np.convolve(x, np.ones(N)/N, mode='same')

%matplotlib widget
plt.close('all')

spd_xyz = np.sqrt(ADCP.isel(bin=0)['X']**2 + ADCP.isel(bin=0)['Y']**2 + ADCP.isel(bin=0)['Z']**2)








# Interpolation of ADCP data in order to match with drone data (data.data) --------------------------------------

#arr_ref = data.data.speed 
#arr2 = spd_xyz
#arr2_interp = interp1d(np.arange(arr2.size),arr2)
#arr2_stretch = arr2_interp(np.linspace(0,arr2.size-1,arr_ref.size))
ADCP_Time=ADCP.time.values.astype('float')

ADCP_Time_interp = interp1d(np.arange(ADCP_Time.size),ADCP_Time)
ADCP_Time_stretch = ADCP_Time_interp(np.linspace(0,ADCP_Time.size-1,data.data.speed.size))

spd_xyz_interp = interp1d(np.arange(spd_xyz.size),spd_xyz)
spd_xyz_stretch = spd_xyz_interp(np.linspace(0,spd_xyz.size-1,data.data.speed.size))

print(len(spd_xyz_stretch))
print(len(data.data.speed))


# Definition of error function
spd_err=smooth(spd_xyz_stretch,50)-data.data.speed





fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
#plt.plot(ADCP_Time_stretch, spd_xyz_stretch, alpha=0.3, color='g',label="ADCP (stretched)")
#plt.plot(data.data['Timestamp'].values.astype('float'),data.data.speed,alpha=0.9,color='k',label="steady flight model")
#plt.plot(data.data['Timestamp'].values.astype('float')[_u],spd_err[_u],alpha=0.9,color='b',label="Speed error (upcast)")
#plt.plot(data.data['Timestamp'].values.astype('float')[_d],spd_err[_d],alpha=0.9,color='r',label="Speed error (downcast)")
plt.scatter(data.data['Timestamp'].values.astype('float')[_u],spd_err[_u], 1,'b', alpha=1,label='upcast')
plt.scatter(data.data['Timestamp'].values.astype('float')[_d],spd_err[_d], 1,'r', alpha=1,label='downcast')
plt.title("speed error")
plt.xlabel("t [s]")
plt.ylabel("v [m/s]")
plt.legend()

t = np.nanpercentile(data.data['Timestamp'].values.astype('float'),[0,100])

def update(x=0):
    ax.set_xlim([t[0] + x*step , t[0] + x*step + step])
    fig.canvas.draw_idle()

ax.set_ylim([-0.5,0.5])
step = np.diff(t)/100

update(30)

plt.show()

1835884
1835884


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
_u = np.remainder(data.data.diveNum,2) == 0
_d = np.remainder(data.data.diveNum,2) == 1

def rmsd(x):
    return np.sqrt(np.nanmean(x**2))
def smooth(x,N):
    return np.convolve(x, np.ones(N)/N, mode='same')

%matplotlib widget
plt.close('all')

spd_xyz = np.sqrt(ADCP.isel(bin=0)['X']**2 + ADCP.isel(bin=0)['Y']**2 + ADCP.isel(bin=0)['Z']**2)








# Interpolation of ADCP data in order to match with drone data (data.data) --------------------------------------
ADCP_Time=ADCP.time.values.astype('float')

ADCP_Time_interp = interp1d(np.arange(ADCP_Time.size),ADCP_Time)
ADCP_Time_stretch = ADCP_Time_interp(np.linspace(0,ADCP_Time.size-1,data.data.speed.size))

spd_xyz_interp = interp1d(np.arange(spd_xyz.size),spd_xyz)
spd_xyz_stretch = spd_xyz_interp(np.linspace(0,spd_xyz.size-1,data.data.speed.size))

print(len(spd_xyz_stretch))
print(len(data.data.speed))


# Definition of error function
spd_err=smooth(spd_xyz_stretch,50)-data.data.speed




#Figure plotting
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
plt.scatter(data.data['Timestamp'].values.astype('float'),spd_err, 1,c=data.data.Temperature,cmap='coolwarm', alpha=1)

cbar = plt.colorbar()
cbar.set_label('Temperature [°C]')

plt.clim([9.5,11])
plt.title("speed error")
plt.xlabel("t [s]")
plt.ylabel("v [m/s]")

t = np.nanpercentile(data.data['Timestamp'].values.astype('float'),[0,100])

def update(x=0):
    ax.set_xlim([t[0] + x*step , t[0] + x*step + step])
    fig.canvas.draw_idle()

ax.set_ylim([-0.5,0.5])
step = np.diff(t)/100

update(30)

plt.show()

1835884
1835884


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [165]:
_u = np.remainder(data.data.diveNum,2) == 0
_d = np.remainder(data.data.diveNum,2) == 1







a=np.diff(data.data.speed)
L=2.7 #length of glider
V=np.nanmean(data.data.speed) #Average speed of drone
v=np.nan_to_num(data.data.speed)
a_max=V*V/(10*L)

validity = np.empty(len(a)+1)
for i in range(len(a)):
    validity[i] = np.sign(a[i]-a_max)

validity_distance = np.empty(len(a)+1)
for i in range(len(a)):
    validity_distance[i] = np.abs(a[i])





def rmsd(x):
    return np.sqrt(np.nanmean(x**2))
def smooth(x,N):
    return np.convolve(x, np.ones(N)/N, mode='same')

%matplotlib widget
plt.close('all')

spd_xyz = np.sqrt(ADCP.isel(bin=0)['X']**2 + ADCP.isel(bin=0)['Y']**2 + ADCP.isel(bin=0)['Z']**2)








# Interpolation of ADCP data in order to match with drone data (data.data) --------------------------------------
ADCP_Time=ADCP.time.values.astype('float')

ADCP_Time_interp = interp1d(np.arange(ADCP_Time.size),ADCP_Time)
ADCP_Time_stretch = ADCP_Time_interp(np.linspace(0,ADCP_Time.size-1,data.data.speed.size))

spd_xyz_interp = interp1d(np.arange(spd_xyz.size),spd_xyz)
spd_xyz_stretch = spd_xyz_interp(np.linspace(0,spd_xyz.size-1,data.data.speed.size))

print(len(spd_xyz_stretch))
print(len(data.data.speed))


# Definition of error function
spd_err=smooth(spd_xyz_stretch,50)-data.data.speed




#Figure plotting
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
plt.scatter(data.data['Timestamp'].values.astype('float'),spd_err, 1,c=validity_distance,cmap='coolwarm', alpha=1)

cbar = plt.colorbar()
cbar.set_label('|a| [m2/s]')

plt.clim([0,0.005])
plt.title("speed error")
plt.xlabel("t [s]")
plt.ylabel("v [m/s]")

t = np.nanpercentile(data.data['Timestamp'].values.astype('float'),[0,100])

def update(x=0):
    ax.set_xlim([t[0] + x*step , t[0] + x*step + step])
    fig.canvas.draw_idle()

ax.set_ylim([-0.5,0.5])
step = np.diff(t)/100

update(30)

plt.show()

1835884
1835884


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …